In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import xraylib
from math import sqrt
import torch
%matplotlib qt 

In [2]:
#dic with energies of elements
dic={ 'Ca':3692,'Mn':5900,'Fe':6405,'Cu':8046,'Au':9713,'Hg':9989,'Pb':10551,'Sr':14165}
num_elements=len(dic)

In [3]:
f = h5py.File('panagia1_mm.h5', 'r')
dset=f['dataset']
dEset=f['energies']
spectra_p=np.array(dset)
energies=np.array(dEset)
f.close()

f = h5py.File('christ.h5', 'r')
dset=f['dataset']
spectra_c=np.array(dset)
f.close()

f = h5py.File('hand.h5', 'r')
dset=f['dataset']
spectra_h=np.array(dset)
f.close()

In [4]:
channels=100
rows_p=41
cols_p=65
rows_c=31
cols_c=46
rows_h=41
cols_h=21

In [5]:
spectra_p = spectra_p.astype(np.float32)
spectra_c = spectra_c.astype(np.float32)
spectra_h = spectra_h.astype(np.float32)
spectra_p=np.delete(spectra_p,np.s_[:channels],1)
spectra_c=np.delete(spectra_c,np.s_[:channels],1)
spectra_h=np.delete(spectra_h,np.s_[:channels],1)

energies=np.delete(energies,np.s_[:channels])

In [44]:
X_h.shape

torch.Size([861, 1948])

In [6]:
#roi function    
def fwhm(ev):
    print(sqrt(2.47*ev+4400))
    return sqrt(2.47*ev+4400)

In [7]:


X_p=torch.tensor(spectra_p)
X_c=torch.tensor(spectra_c)
X_h=torch.tensor(spectra_h)

In [8]:

key_list=list(dic.keys())
min_ch=[]
max_ch=[]
min_roi_all=[]
max_roi_all=[]
for element in dic:
    min_roi=(dic[element]-fwhm(dic[element])/2)
    min_roi_all.append(min_roi)
    max_roi=(dic[element]+fwhm(dic[element])/2)
    max_roi_all.append(max_roi)
    min_ch.append(int((min_roi+960)/20)-channels)
    max_ch.append(int((max_roi+960)/20)-channels)
print(min_ch)
print(max_ch) 

116.27226668470861
116.27226668470861
137.7425134081704
137.7425134081704
142.1982770641051
142.1982770641051
155.7999358151344
155.7999358151344
168.49661717672555
168.49661717672555
170.50756581454092
170.50756581454092
174.53071362943544
174.53071362943544
198.46296883801776
198.46296883801776
[129, 239, 264, 346, 429, 443, 471, 651]
[135, 246, 271, 354, 437, 451, 479, 661]


In [114]:
spectra_h.shape

(861, 1948)

In [9]:
spectra_p_cube=spectra_p.reshape(rows_p,cols_p,spectra_p.shape[1])
spectra_c_cube=spectra_c.reshape(rows_c,cols_c,spectra_c.shape[1])
spectra_h_cube=spectra_h.reshape(cols_h,rows_h,spectra_h.shape[1])

In [125]:
%matplotlib qt
f1=plt.figure()
plt.imshow(np.sum(spectra_h_cube[:,:,min_ch[2]:max_ch[2]],axis=2))
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()
plt.rcParams.update({'font.size': 8})

In [10]:
y_p=np.zeros([len(spectra_p),num_elements])
for i in range(len(spectra_p)):
    for j in range(len(key_list)):
        y_p[i][j]=np.sum(np.sum(spectra_p[i,min_ch[j]:max_ch[j]]))
y_c=np.zeros([len(spectra_c),num_elements])
for i in range(len(spectra_c)):
    for j in range(len(key_list)):
        y_c[i][j]=np.sum(np.sum(spectra_c[i,min_ch[j]:max_ch[j]]))
y_h=np.zeros([len(spectra_h),num_elements])
for i in range(len(spectra_h)):
    for j in range(len(key_list)):
        y_h[i][j]=np.sum(np.sum(spectra_h[i,min_ch[j]:max_ch[j]]))

In [11]:
y_p_cube=y_p.reshape(rows_p,cols_p,num_elements)
y_c_cube=y_c.reshape(rows_c,cols_c,num_elements)
y_h_cube=y_h.reshape(cols_h,rows_h,num_elements)

In [12]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(hidden_layer_sizes=(100,),activation='relu',solver='adam',verbose=True,max_iter=500)



In [13]:
regr.fit(X_h, y_h)


Iteration 1, loss = 8258.49851417
Iteration 2, loss = 6266.53834506
Iteration 3, loss = 4618.82846787
Iteration 4, loss = 3211.04459452
Iteration 5, loss = 2140.27677113
Iteration 6, loss = 1358.00062015
Iteration 7, loss = 847.54109457
Iteration 8, loss = 587.78388711
Iteration 9, loss = 471.81187792
Iteration 10, loss = 401.35775360
Iteration 11, loss = 321.65105503
Iteration 12, loss = 249.29700313
Iteration 13, loss = 200.83260531
Iteration 14, loss = 166.68337772
Iteration 15, loss = 132.30865162
Iteration 16, loss = 110.20620172
Iteration 17, loss = 92.44517698
Iteration 18, loss = 79.16274224
Iteration 19, loss = 69.22756525
Iteration 20, loss = 61.74514374
Iteration 21, loss = 56.51142849
Iteration 22, loss = 52.06902907
Iteration 23, loss = 48.07013131
Iteration 24, loss = 44.49752408
Iteration 25, loss = 41.31159918
Iteration 26, loss = 38.69481845
Iteration 27, loss = 36.29052943
Iteration 28, loss = 33.80575660
Iteration 29, loss = 31.91946685
Iteration 30, loss = 29.963886

Iteration 251, loss = 0.62100395
Iteration 252, loss = 0.60129300
Iteration 253, loss = 0.60821377
Iteration 254, loss = 0.60378824
Iteration 255, loss = 0.60680711
Iteration 256, loss = 0.58957090
Iteration 257, loss = 0.58031929
Iteration 258, loss = 0.56611272
Iteration 259, loss = 0.56104230
Iteration 260, loss = 0.56240424
Iteration 261, loss = 0.56088498
Iteration 262, loss = 0.54747144
Iteration 263, loss = 0.53552127
Iteration 264, loss = 0.53501095
Iteration 265, loss = 0.52668839
Iteration 266, loss = 0.53021598
Iteration 267, loss = 0.53148185
Iteration 268, loss = 0.52467050
Iteration 269, loss = 0.49880604
Iteration 270, loss = 0.48984604
Iteration 271, loss = 0.48578257
Iteration 272, loss = 0.49031215
Iteration 273, loss = 0.48558725
Iteration 274, loss = 0.48137201
Iteration 275, loss = 0.46024088
Iteration 276, loss = 0.46062971
Iteration 277, loss = 0.47005202
Iteration 278, loss = 0.45749478
Iteration 279, loss = 0.46627666
Iteration 280, loss = 0.44396533
Iteration 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(max_iter=500, verbose=True)

In [50]:
plt.plot(regr.loss_curve_[20:])

In [119]:
regr.fit(X_p, y_p)

Iteration 1, loss = 29065.32217228
Iteration 2, loss = 13611.43165639
Iteration 3, loss = 6307.96675691
Iteration 4, loss = 2967.18782684
Iteration 5, loss = 1182.17284455
Iteration 6, loss = 432.57502525
Iteration 7, loss = 257.16810740
Iteration 8, loss = 190.75018614
Iteration 9, loss = 140.40339158
Iteration 10, loss = 109.43844617
Iteration 11, loss = 90.71681345
Iteration 12, loss = 77.93291224
Iteration 13, loss = 69.64514239
Iteration 14, loss = 62.54706842
Iteration 15, loss = 56.89507483
Iteration 16, loss = 51.64912719
Iteration 17, loss = 47.54488944
Iteration 18, loss = 44.23811090
Iteration 19, loss = 41.85407203
Iteration 20, loss = 39.67580951
Iteration 21, loss = 38.10533640
Iteration 22, loss = 36.51889163
Iteration 23, loss = 35.38178581
Iteration 24, loss = 34.35117407
Iteration 25, loss = 33.39922241
Iteration 26, loss = 32.85181279
Iteration 27, loss = 32.07761387
Iteration 28, loss = 31.61805354
Iteration 29, loss = 30.94990335
Iteration 30, loss = 30.46206949
It

Iteration 249, loss = 1.97374908
Iteration 250, loss = 1.80260729
Iteration 251, loss = 1.52801543
Iteration 252, loss = 1.60893139
Iteration 253, loss = 1.42187953
Iteration 254, loss = 1.37464083
Iteration 255, loss = 1.45919747
Iteration 256, loss = 1.46823355
Iteration 257, loss = 1.44616512
Iteration 258, loss = 1.34386678
Iteration 259, loss = 1.39270730
Iteration 260, loss = 1.46362961
Iteration 261, loss = 1.40557073
Iteration 262, loss = 1.32905669
Iteration 263, loss = 1.25995656
Iteration 264, loss = 1.35756562
Iteration 265, loss = 1.48993701
Iteration 266, loss = 1.31163024
Iteration 267, loss = 1.29972133
Iteration 268, loss = 1.24237943
Iteration 269, loss = 1.23917824
Iteration 270, loss = 1.23895331
Iteration 271, loss = 1.31151344
Iteration 272, loss = 1.30247763
Iteration 273, loss = 1.23936283
Iteration 274, loss = 1.24633557
Iteration 275, loss = 1.18500427
Iteration 276, loss = 1.18849133
Iteration 277, loss = 1.10704061
Iteration 278, loss = 1.09507999
Iteration 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(max_iter=300, verbose=True)

In [14]:
a=regr.predict(X_p)

In [12]:
print(X_c.shape)
print(X_p.shape)

torch.Size([1426, 1948])
torch.Size([2665, 1948])


In [21]:
print(a[1],y_p[1])

[ 40.81013896 183.08454245   6.35860781   8.13479305 112.96478304
   6.73762444] [ 38.69000244 182.94000244   8.98999977   6.11000013 108.02000427
   4.97000027]


In [31]:
b=a.reshape(rows_p,cols_p,num_elements)
for i in range(num_elements):
    f2=plt.figure()
    plt.imshow(b[:,:,i])
#plt.title()
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.title(key_list[i])

In [16]:
a.shape

(2665, 8)

In [30]:
min_cu=int(np.min(a[:,3]))
max_cu=int(np.max(a[:,3]))
window_size=int(max_cu-min_cu)//100
counts=np.zeros(100)
for i in range(len(X_p)):
    

19
